# CMSC320 Final Project
## Kinsey Smith, Sarah Bullard, Yiwen Shen

### TODO - explanation of what we did and how we did it, and how successful we ended up being. 

In [17]:
import tweepy
import textblob
import numpy as np

In order to access Twitter's API, we had to create applications and personally get authentication tokens. Even though we are all allowed access to Twitter, we cannot give out these confidential tokens on this public notebook. In order to get past this hurdle, we created a function that would pull from our own files on our own machines for these tokens. In the cell below is a copy of credentials.py, without the confidential information. 

In [18]:
from credentials import *

In [19]:
#A function that takes these credentials and sets up the API.
def api_setup():
    authentication = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    authentication.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
    api = tweepy.API(authentication)
    return api

In [20]:
# Extracting the tweets
extract_tweets = api_setup()

In [21]:
tweets = extract_tweets.user_timeline(screen_name="realDonaldTrump")

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [ ]:
# First, get the tweet ids of all of the donald trump statuses for the past determined amount of time (2 years?)
# Then, get all of the tweets that have replied to these tweet ids (in tweety, )
# get the id of donald trump. for now, i'll use 1234.